This is the script create LiDAR-pkl and anno-png for nuscene. It is the same format as the waymo dataset we produced form clft and clfcn training. Camera images are in ordinary png format, LiDAR should be a pkl file contains the two dicts 'camera_coordinates' and '3d_points' that represent the information of points in camera view. 

In waymo dataset, the pkl files contains all points, but the camera-view-points has the first number in 'camera_coordinates' as 1 (so not-in-camera-view-points have 0). 'camera_coordinates' and '3d_points' are in same shape, so you can find the pair of 'camera_coordinates' and '3d_points' correspondingly. For annotation, it is a png file which is the lidar-porjection. Becuase in waymo the label is based on the lidar points, so the pixels correspond to lidar points have specific values for annotation (1 for car, 2 for human, etc. 0 is for nothing.)

The goal is to make the nuscene data the same way as our waymo data. Here use the nuscene-devkit. It has the functions to render the lidar points on different camera views, also lidar points are colored based on the classes. We took the related functions out and return the variables we needed. 

In [1]:
import json
import math
import os
import os.path as osp
import sys
import pickle
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
from PIL import Image

### This is the important part, the 'nuscene_lidar_render.py' script is in same folder. It contanils the modified functions.

In [2]:
from nuscene_lidar_render import NuScenes

In [3]:
nusc = NuScenes(version='v1.0-trainval', dataroot='/home/autolab/Data/nuscene/', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
Loading nuScenes-lidarseg...
Loading nuScenes-panoptic...
32 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
34149 lidarseg,
34149 panoptic,
Done loading in 29.917 seconds.
Reverse indexing ...
Done reverse indexing in 8.2 seconds.


In [4]:
nusc.scene

[{'token': '73030fb67d3c46cfb5e590168088ae39',
  'log_token': '6b6513e6c8384cec88775cae30b78c0e',
  'nbr_samples': 40,
  'first_sample_token': 'e93e98b63d3b40209056d129dc53ceee',
  'last_sample_token': '40e413c922184255a94f08d3c10037e0',
  'name': 'scene-0001',
  'description': 'Construction, maneuver between several trucks'},
 {'token': 'c3e0e9f6ee8d4170a3d22a6179f1ca3a',
  'log_token': '6b6513e6c8384cec88775cae30b78c0e',
  'nbr_samples': 40,
  'first_sample_token': '9e28820bd2ba4eb480e1b9079f17c30c',
  'last_sample_token': 'd0dc3c18dfc64c779bc9e4ee6e9e2e3b',
  'name': 'scene-0002',
  'description': 'Intersection, peds, waiting vehicle, parked motorcycle at parking lot'},
 {'token': 'e7ef871f77f44331aefdebc24ec034b7',
  'log_token': '92af2609d31445e5a71b2d895376fed6',
  'nbr_samples': 40,
  'first_sample_token': 'fd8420396768425eabec9bdddf7e64b6',
  'last_sample_token': '20684b77c01641e7a5ada48308a0b1e1',
  'name': 'scene-0003',
  'description': 'Parking lot, barrier, exit parking lot

### Now to check and distinguish the dataset by weather. day-fair:602 day-rain:149 night-fair:83 night-rain:16
### So in total they are 850, which is correct. 

In [15]:
# Find the scene day-fair  total 602
n=0
for i in nusc.scene:
    if ('Night' not in i['description']) & ('rain' not in i['description']) & ('wet' not in i['description']) & ('Rain' not in i['description']) & ('Wet' not in i['description']):
        print(i)
        n+=1
        print(n)

{'token': '73030fb67d3c46cfb5e590168088ae39', 'log_token': '6b6513e6c8384cec88775cae30b78c0e', 'nbr_samples': 40, 'first_sample_token': 'e93e98b63d3b40209056d129dc53ceee', 'last_sample_token': '40e413c922184255a94f08d3c10037e0', 'name': 'scene-0001', 'description': 'Construction, maneuver between several trucks'}
1
{'token': 'c3e0e9f6ee8d4170a3d22a6179f1ca3a', 'log_token': '6b6513e6c8384cec88775cae30b78c0e', 'nbr_samples': 40, 'first_sample_token': '9e28820bd2ba4eb480e1b9079f17c30c', 'last_sample_token': 'd0dc3c18dfc64c779bc9e4ee6e9e2e3b', 'name': 'scene-0002', 'description': 'Intersection, peds, waiting vehicle, parked motorcycle at parking lot'}
2
{'token': 'e7ef871f77f44331aefdebc24ec034b7', 'log_token': '92af2609d31445e5a71b2d895376fed6', 'nbr_samples': 40, 'first_sample_token': 'fd8420396768425eabec9bdddf7e64b6', 'last_sample_token': '20684b77c01641e7a5ada48308a0b1e1', 'name': 'scene-0003', 'description': 'Parking lot, barrier, exit parking lot'}
3
{'token': '25496f19ffd14bd088cb4

In [16]:
# Find the scene day-rain total 149
n=0
for i in nusc.scene:
    if ('Night' not in i['description']) & (('wet' in i['description']) | ('rain' in i['description']) | ('Rain' in i['description']) | ('Wet' in i['description'])):
        n+=1
        print(i)
        print(n)

{'token': '64e1ad8a976542ab86c427d714550d06', 'log_token': 'b2685a235700404581dc7354dd5b4eda', 'nbr_samples': 41, 'first_sample_token': '0dcd66cdeafb4978be2a580289336198', 'last_sample_token': 'bf4792be43ae41b78d8de96e0d07679b', 'name': 'scene-0440', 'description': 'Rain, small crane, parking lot'}
1
{'token': '93481c2504474ca0b21d4abbab2cf3cf', 'log_token': 'b2685a235700404581dc7354dd5b4eda', 'nbr_samples': 40, 'first_sample_token': '604726ccdcb64dc09b189c3edf31aec2', 'last_sample_token': '0fc008ed23194ee5b5b3b54983c7a9fd', 'name': 'scene-0441', 'description': 'Rain, cross intersection, lane change, parking lot'}
2
{'token': 'e54f4a36a8b94aebbe4189186dee8f05', 'log_token': 'b2685a235700404581dc7354dd5b4eda', 'nbr_samples': 41, 'first_sample_token': 'f29e030d1be54060a3182f37b701ac12', 'last_sample_token': 'd849725384014d29b12bd2652ef3fae0', 'name': 'scene-0442', 'description': 'Rain, parked truck trailers, parked cars'}
3
{'token': '6193fac132614eb1a09dcbf3f359460f', 'log_token': 'b268

In [17]:
# Find the scene night-fair  total 83
n=0
for i in nusc.scene:
    if ('Night' in i['description']) & ('rain' not in i['description']) & ('wet' not in i['description']) & ('Rain' not in i['description']) & ('Wet' not in i['description']) :
        n+=1
        print(i)
        print(n)

{'token': 'ca6abd081eaf48689f06b5e8fcc9d369', 'log_token': '6175c3299ae0482791f4ab1e9d54b326', 'nbr_samples': 41, 'first_sample_token': '8b6c00e4e92c4fe398d2719ea7cd20e0', 'last_sample_token': 'd7a9b9d627384700b888a3ea48e04de0', 'name': 'scene-0992', 'description': 'Night, ped in dark, parked cars'}
1
{'token': 'e71f69d57d944b6cb0ffc516a88b6104', 'log_token': '6175c3299ae0482791f4ab1e9d54b326', 'nbr_samples': 40, 'first_sample_token': '0e449f05cf5941aa9965b215e6d0b346', 'last_sample_token': '981047eba7c74fe28116bb583bf2441b', 'name': 'scene-0994', 'description': 'Night, bump, ped in dark, low traffic, roundabout'}
2
{'token': 'c6d1085c40614693a046dae3168bdb35', 'log_token': '6175c3299ae0482791f4ab1e9d54b326', 'nbr_samples': 40, 'first_sample_token': '9711d62fa66f40a9894043673518b95f', 'last_sample_token': 'a86def9a6d804c47b143f9fc7a40ae67', 'name': 'scene-0995', 'description': 'Night, ped, bus, slope, flyover'}
3
{'token': '796e7ae080984a0ab2b96ce80f18ca3b', 'log_token': '6175c3299ae04

In [18]:
# Find the scene night-rain total 16
n=0
for i in nusc.scene:
    if ('Night' in i['description']) & (('wet' in i['description']) | ('rain' in i['description']) | ('Rain' in i['description']) | ('Wet' in i['description'])) :
        n+=1
        print(i)
        print(n)

{'token': '4ed628299b1e45a3a73704e8cf8287a9', 'log_token': '853a9f9fe7e84bb8b24bff8ebf23f287', 'nbr_samples': 41, 'first_sample_token': 'cb4527f15788499ab7c61439087f823c', 'last_sample_token': 'd054306ed5c64d8a9621d9c00b551374', 'name': 'scene-1053', 'description': 'Night, rain, bump, peds, congestion, parked car'}
1
{'token': '7052d21b95fc4bae8761b8d9524f3e42', 'log_token': 'e55205b1f2894b49957905d7ddfdb96d', 'nbr_samples': 40, 'first_sample_token': 'e74b8ce0b1824e0c8167f2534d8d4f7d', 'last_sample_token': '2deffe9affa24bb2ae3f6f440cb8a01e', 'name': 'scene-1060', 'description': 'Night, rain, turn, ped in dark'}
2
{'token': 'bd338b912ce9434995b29b6dac9fbf1d', 'log_token': 'e55205b1f2894b49957905d7ddfdb96d', 'nbr_samples': 40, 'first_sample_token': 'dc2def2a7bc94693b67eba2ffbe13a56', 'last_sample_token': '0888060b915444b592ab9ae2527b9d80', 'name': 'scene-1065', 'description': 'Rain, Night, bump, motorcycle, difficult lighting, overtake parked car'}
3
{'token': '16be583c31a2403caa6c158bb5

### Now dealing with 16 night-rain scenes

In [23]:
for i in nusc.scene:
    if ('Night' in i['description']) & (('wet' in i['description']) | ('rain' in i['description']) | ('Rain' in i['description']) | ('Wet' in i['description'])):
        scene_token = i['token']
        print(scene_token)

4ed628299b1e45a3a73704e8cf8287a9
7052d21b95fc4bae8761b8d9524f3e42
bd338b912ce9434995b29b6dac9fbf1d
16be583c31a2403caa6c158bb55ae616
865c607c8ef44f13b39744a0de110740
26c089d84086473e87607ae5c6ae85c6
de7d80a1f5fb4c3e82ce8a4f213b450a
a2b64d02c5fa4b9bab671a97561b6b33
cfa36eca40364e5bb15f550a077a21c5
04e5f089805540a5a9e73c1b7c54ee8c
e3b6fe9201c64334be311d4249a0c80e
e5514b8f435e475cac4eba74b44773b5
89f20737ec344aa48b543a9e005a38ca
d3c39710e9da42f48b605824ce2a1927
7ed1b613f0ee47ceaf6d19ca3947f825
57146b2ebf10432f84b15a3038fe1755


In [24]:
scene_token

'57146b2ebf10432f84b15a3038fe1755'

In [105]:
scene_record = nusc.get('scene', scene_token)
total_num_samples = scene_record['nbr_samples']
first_sample_token = scene_record['first_sample_token']
last_sample_token = scene_record['last_sample_token']


current_sample_token = first_sample_token
keep_looping = True
i = 0


while keep_looping:
    if current_sample_token == last_sample_token:
        keep_looping = False
        
    sample_record = nusc.get('sample', current_sample_token)
    
    camera_token = sample_record['data']['CAM_FRONT']
    cam = nusc.get('sample_data', camera_token)
    filename = scene_record['name'] + '_' + 'CAM_FRONT' + '_' + os.path.basename(cam['filename'])
  
    pc, points, d3_points, coloring, im = nusc.render_pointcloud_in_image(sample_record['token'],
                                pointsensor_channel='LIDAR_TOP',
                                camera_channel='CAM_FRONT',
                                render_intensity=False,
                                show_lidarseg=True,                                 
                                filter_lidarseg_labels=[2, 3, 4, 5, 6, 7, 8, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
                                show_lidarseg_legend=False)
    
    
    
    
    next_sample_token = sample_record['next']
    current_sample_token = next_sample_token
    i+=1
# print(sample_record)

FileNotFoundError: [Errno 2] No such file or directory: '/home/autolab/Data/nuscene/samples/LIDAR_TOP/n008-2018-09-18-14-35-12-0400__LIDAR_TOP__1537296075899673.pcd.bin'

In [97]:
camera_token = sample_record['data']['CAM_FRONT']
camera_token

'55d28a7694c442529924a47298825246'

In [100]:
cam = nusc.get('sample_data', camera_token)
cam

{'token': '55d28a7694c442529924a47298825246',
 'sample_token': 'b8f4bec20b9b4d79b1979f43ba1e0222',
 'ego_pose_token': '55d28a7694c442529924a47298825246',
 'calibrated_sensor_token': '2316cc74012f44a2b730660925f5b926',
 'timestamp': 1537296095512404,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n008-2018-09-18-14-35-12-0400__CAM_FRONT__1537296095512404.jpg',
 'prev': 'a6936b9cb47b4b1f90e9fb983525d248',
 'next': '',
 'sensor_modality': 'camera',
 'channel': 'CAM_FRONT'}

In [102]:
filename = scene_record['name'] + '_' + 'CAM_FRONT' + '_' + os.path.basename(cam['filename'])
filename

'scene-0635_CAM_FRONT_n008-2018-09-18-14-35-12-0400__CAM_FRONT__1537296095512404.jpg'

In [85]:
scene_record

{'token': '7bd098ac88cb4221addd19202a7ea5de',
 'log_token': '01ebe4444f624b4699ec6bf58837b153',
 'nbr_samples': 40,
 'first_sample_token': '43f05ac397484672b9bdf86dcc0b0289',
 'last_sample_token': 'b8f4bec20b9b4d79b1979f43ba1e0222',
 'name': 'scene-0635',
 'description': 'Stationary in rain, semi backing up'}

In [86]:
total_num_samples

40

In [82]:
current_token

'763d88492862492185d8ae1cf890cc9e'